In [7]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from src.call_finder import *

# Community Peer Explorer Demo

This notebook is an interactive demo of the Locus Peer Explorer package. 
It allows you to find and explore the peers of any MSA in 2015 or 2016. 

### Geographic Peers

The first type of peer we can identify are geographic peers.
This feature returns the MSA which are the closest to you.

In [2]:
geo = interact_manual(show_peers, df_data=fixed(df_data), df_county_dist=fixed(df_county_dist), df_msa_def= fixed(df_msa_def),msa=input_msa, n_peers=input_n_peers, year=fixed(2015))


interactive(children=(Dropdown(description='MSA', index=237, layout=Layout(width='80%'), options={'Abilene, TX…

### Input FMs peers

This second feature allows the user to find its peers based on any subset of FMs. This will return the MSAs that have the most similar concentration of those FMs. You can also choose to identify peers based on outcomes.

To select multiple FMs or outcomes, hold the cmd key.

The graphs allow to compare:
* the FM concentration of the user MSA to its peers, to the national mean and 1st and 3rd quantile (in red).
* the distribution of FM concentration within the peer group and among all other MSAs.

In [3]:
input_peers = interact_manual(show_fms_peers, df_data=fixed(df_data),msa=input_msa, year=fixed(2015), n_peers=input_n_peers,fms=input_fms, outcomes=input_outcomes)


interactive(children=(Dropdown(description='MSA', index=237, layout=Layout(width='80%'), options={'Abilene, TX…

## Automatically selected FMs

For all the next appoaches, we automatically select a subset of FMs or Outcomes and then find the MSAs that are the most similar to you on this subset of features using a distance metric. 

#### Most present FMs 


This feature will return the peers based on the first top n FMs that are the most prevalent in your MSA. You will be able to specify n, the number of FMs you want to base the peer identification on.
  

In [4]:
top_fms = interact_manual(show_top_fms_peers, df_data=fixed(df_data),msa=input_msa, year=input_year, n_peers=input_n_peers, n_fms=input_n_fms)


interactive(children=(Dropdown(description='MSA', index=237, layout=Layout(width='80%'), options={'Abilene, TX…

#### Distinguishing features 

The peers returned here are based on the FMs that are distinguishing for your MSA. It will include the most present FMs but also the least present. FMs are ranked based on how much the concentration in your MSA differs from other MSAs. You can choose how many of those distinguishing features you want to base your analysis on.

In [5]:
disting = interact_manual(show_disting_peers, df_data=fixed(df_data), msa=input_msa, year=input_year, n_peers=input_n_peers, n_feat=input_n_features)


interactive(children=(Dropdown(description='MSA', index=237, layout=Layout(width='80%'), options={'Abilene, TX…

#### Minimum coverage peers

This features selects enough FMs so that a given percentage of the total employment in your MSA is covered. As before, the FMs are selected from the most present to the least present.


In [6]:
cover = interact_manual(show_coverage_peers, df_data=fixed(df_data), msa=input_msa, year=input_year, n_peers=input_n_peers, coverage=input_coverage)


interactive(children=(Dropdown(description='MSA', index=237, layout=Layout(width='80%'), options={'Abilene, TX…

### To do 

                                                                                                                                                                                                                                         